In [1]:
!pip install librosa

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Processing /home/hanna/.cache/pip/wheels/6a/ae/a3/9926619fb71320153272a879bcae86f4b1e813b5da5e482a80/librosa-0.7.2-py2-none-any.whl
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Using cached scikit_learn-0.20.4-cp27-cp27mu-manylinux1_x86_64.whl (5.5 MB)
Processing /home/hanna/.cache/pip/wheels/19/90/bb/f45e7cc1cfc8f5299c12511fccefcca90c801de995a4e7eb00/resampy-0.2.2-py2-none-any.whl
  Using cached numba-0.47.0-cp27-cp27mu-manylinux1_x86_64.whl (3.6 MB)
Processing /home/ha

In [8]:
import os
os.mkdir('data/')

In [3]:

import numpy as np
import librosa


def extract_spectrum(file_name, file_length=5, bin_factor=4, **kwargs):
    """
    file_length is the target time length of the file in seconds
    bin_factor is the amount by which we enlarge the time bins from 512 samples:
    Extract mel spectrogram from audio file `file_name`
    """

    X, sample_rate = librosa.core.load(file_name)
    file_name = file_name.split('/')[-1]
    target_name = file_name[:-4]
    target_length = int(file_length * sample_rate)
    target = np.zeros(target_length)
    if X.shape[0] >= target_length:
        target = X[:target_length]
    else:
        target[:X.shape[0]] = X
    X = target
    mel = librosa.feature.melspectrogram(X, sr=sample_rate, hop_length=512 * bin_factor).T
    print(file_name, ' loaded with shape ', mel.shape)
    return mel
#     loadname = './data/' + target_name
#     if not Path(loadname).is_file():
#       np.save(loadname, mel)


In [4]:
# import os 

# cur_dir = './pygender/test_data/AudioSet/'
# genders = ['female', 'male']
# data = []

# for gen in genders:
#     data_dir = cur_dir + (str(gen) + '_clips/')
#     for filename in os.listdir(data_dir):
#         extract_spectrum(data_dir + filename)
#         data.append(('data/'+filename, gen))

In [11]:
# import pandas as pd 

# df = pd.DataFrame(data, columns = ['filename', 'gender'])

# df.head()

,filename,gender
0,data/1SCwc3vQsWA.wav,female
1,data/35KLb0tUd0U.wav,female
2,data/-IAuM2eOMKI.wav,female
3,data/-tN-vxr1HjM.wav,female
4,data/6FpFqBxK1fE.wav,female


In [8]:
!pip3 install SpeechRecognition

Defaulting to user installation because normal site-packages is not writeable
  Using cached SpeechRecognition-3.8.1-py2.py3-none-any.whl (32.8 MB)


In [12]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import speech_recognition as sr
from tensorflow.keras.models import load_model
#from text_to_speech import get_large_audio_transcription
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [10]:
LABEL2INT = {'female':0,
            'male':1}

INT2LABEL = {0: 'female',
             1: 'male'}

shape = (54, 128,1)


In [13]:
reconstructed_model = load_model("Daniel/best_model_final.h5")

testfile = extract_spectrum('hanna/gettysburg10.wav').reshape(1,*shape)


reconstructed_model.predict(testfile)

gettysburg10.wav  loaded with shape  (54, 128)


array([[0.28677973]], dtype=float32)

In [ ]:
# import os
# from flask import Flask, request, jsonify
# from tensorflow.keras.models import load_model
# import numpy as np
# import json

# app = Flask(__name__) 

# def open_model(filename = 'best_model_final.h5'):
    
#     """ Given filename, returns the model 
#     param filename: filepath for model
#     return : pretrained model """

#     return load_model(filename)

# def predict_from_model(X):
    
#     """ Given X, returns a prediction 
#     param X: the feature values to consider
#     returns pred_pickle: the prediction """
#     model = open_model()
#     if not isinstance(X, np.ndarray):
#         X = np.array(eval(str(X)))
#     if len(X.shape) == 1:
#         X = X[np.newaxis , :]  
#     pred_pickle = model.predict(X)
#     return str(pred_pickle[0:X.shape[0]])

# @app.route('/')
# def enter():
#     return 'Welcome!'
# @app.route('/predict_single')
# def predict_single():
#     """ Returns prediction for given parameters"""
#     pred_array = np.zeros(len(cols))
    
#     for i, col in enumerate(cols):
#         pred_array[i] = request.args.get(col)
#     try:
#         return use_pickle(pred_array)
#     except:
#         return use_pickle(np.zeros(len(cols)))
# @app.route('/predict', methods=['POST'])
# def predict():
#     data = request.get_json()
#     prediction = use_pickle(data)

#     return jsonify(prediction)


# if __name__ == "__main__":
#     global model
#     model = open_model()
#     port = os.environ.get('PORT')
#     if port:
#         app.run(host='0.0.0.0', port=int(port))
#     else:
#         app.run()

In [103]:
!pip install pydub

In [104]:
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr


r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                whole_text += text
    # return the text for all chunks detected
    return whole_text


In [117]:
import re
def get_translation(output, gender):
    heb_I =  'אני'
    heb_she = 'היא'
    heb_he = 'הוא'
    ind = []
    translate = ''
    lenght = 0
    translator = Translator()
    replacer = 'I'
    

    for sentence, val in output:
        res = translator.translate(sentence, dest='he').text
        res_list = re.findall(r"[\w]+|['.,!?;]", res)
        if val>0:
          if gender == 'female':
            replacer = 'she'
            heb_replacer = heb_she
          elif gender == 'male':
            replacer = 'he'
            heb_replacer = heb_he
          index = [i for i,x in enumerate(res_list) if x==heb_I][0]
          ind.append(index+lenght) 
        lenght += len(res_list) 
        sentence_list = re.findall(r"[\w]+|['.,!?;]", sentence)
        sentence = ' '.join([replacer if word == 'I' else word for word in sentence_list])
        translate += " " + sentence
    gender_res = translator.translate(translate, dest='he').text
    gender_res_list = re.findall(r"[\w]+|['.,!?;]", gender_res)
    for i in ind:
        if gender_res_list[i] ==  heb_replacer:
          gender_res_list[i] = heb_I
        elif gender_res_list[i-1] ==  heb_replacer:
          gender_res_list[i-1] = heb_I
        elif gender_res_list[i+1] ==  heb_replacer:
          gender_res_list[i+1] = heb_I
    print("""\n\nGender Change : {} : {} => {}""".format(
                                                    gender,
                                                    ' '.join([x[0] for x in output]),
                                                    ' '.join(gender_res_list))
                                                    )
    return ' '.join(gender_res_list)

In [126]:
import spacy
nlp = spacy.load('en')

present_tenses = ['VBP', 'VBZ','VBG']


def determine_chunks(pronouns, vals, text, 
                     present_tenses = present_tenses):
    
    chunks = []
    
    iterations = len(pronouns)
    
    for i in range(iterations):
        
        start = vals[i]
        stop = None 
        
        if i != (iterations - 1):
            stop = vals[i+1]

        if pronouns[i] != 'I':
            chunks.append((str(text[start:stop]),0))
            continue
            

        present_verbs_change = [word for word in text[start:stop]
                                if word.tag_ in
                                present_tenses]

        if len(present_verbs_change) == 0:
            chunks.append((str(text[start:stop]),0))
            continue

        chunks.append((str(text[start:stop]),1))
        
    return chunks

In [132]:
from nltk import word_tokenize, pos_tag


def determine_tense_input(sentence, present_tenses = present_tenses): 
    
    """ Seperates words into chunks based on subjects of the sentence
    and sends them to the determine_chunks function to determine
    if that chunk requires further manipulation or not 
    param ....
    .....
    ....
    returns: list of tuples (portion, int) where 1 means the chunk
    requires further manipulation and 0 means it can be left as is """

    text = nlp(sentence)
    
    present = len([word for word in text 
                    if word.tag_ in present_tenses])
    
    if present == 0 :
        return [(sentence,0)]
    
    sub_toks = list(sorted([ (i, tok) for i, tok 
                            in enumerate(text)
                            if tok.dep_ == "nsubj" ]))


    vals = [x[0] for x in sub_toks]
    pronouns = [str(x[1]) for x in sub_toks]
    
    if 'I' not in pronouns:
        return [(sentence,0)]
            
    return determine_chunks(pronouns, vals, text)
  

tuple

In [122]:
#!pip install googletrans

In [133]:
from googletrans import Translator

### THIS NEEDS TO CHANGE FOR INFERENCE MAIN
### CURRENTLY IT'S ASSUMING THERE ARE TEST FOLDERS


test_ind = data['indices_test'][0]

test_file , test_gender = df.iloc[test_ind]['filename'], df.iloc[test_ind]['gender']

gend_folder = 'female_clips/'
if test_gender =='male':
  gend_folder = 'male_clips/'

test_file = 'pygender/test_data/AudioSet/' + gend_folder + test_file.split('/')[-1]

sing_shape = data['X_test'][0].shape
test_vals = data['X_test'][0].reshape(1,*sing_shape)

pred = model.predict(test_vals)
pred = pred[0][0]

print('Prediction : ', INT2LABEL[round(pred)])
print('True : ', test_gender)
print('__________________')
chunks = get_large_audio_transcription(test_file)

print('Audio : \n') #,chunks)

split_sentence = determine_tense_input(chunks)
print(split_sentence)
print(get_translation(split_sentence, test_gender))
#print('\nAudio : \n\n',get_large_audio_transcription(te))

Prediction :  female
True :  male
__________________
Audio : 

[("For european race driver it seems like an impossible goal to become a nascar champion. Still a while ago we simply doesn't have a nascar championship in europe. ", 0)]


Gender Change : male : For european race driver it seems like an impossible goal to become a nascar champion. Still a while ago we simply doesn't have a nascar championship in europe.  => עבור נהג המירוצים האירופי זה נראה כמו מטרה בלתי אפשרית להפוך לאלוף נשק . עדיין לפני זמן מה פשוט אין לנו אליפות נשק באירופה .
עבור נהג המירוצים האירופי זה נראה כמו מטרה בלתי אפשרית להפוך לאלוף נשק . עדיין לפני זמן מה פשוט אין לנו אליפות נשק באירופה .
